In [ ]:
!nvidia-smi

In [1]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           62Gi       719Mi        33Gi       0.0Ki        29Gi        61Gi
Swap:         8.0Gi       263Mi       7.7Gi


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Dense, Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from sklearn.utils import class_weight
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from statistics import mode
from tqdm import tqdm, trange
from keras.models import load_model
import keras
import soundtransfer
from tcn import TCN, tcn_full_summary
from sklearn.utils import shuffle
import gc

In [3]:
sfreq = 16000

In [4]:
def concat(lol):
    start = lol[0]
    for i in range(1, len(lol)):
        start = list(start) + list(lol[i])
    return start

In [5]:
feature_mode = 'time_seg'
mode = 'train'

# features = []
labels = []
for i in range(3):
#     features_part = pickle.load(open('features/{}/{}_features_part{}.pkl'.format(feature_mode, mode, i+1), 'rb'))
#     features.append(features_part)
    labels_part = pickle.load(open('features/{}/{}_labels_part{}.pkl'.format(feature_mode, mode, i+1), 'rb'))
    labels.append(labels_part)
    
# features = concat(concat(features))
labels = concat(concat(labels))

k=0
for i in range(len(labels)-k):
    if labels[i-k] == []:
#         del features[i-k]
        del labels[i-k]
        k+=1

In [6]:
labels_flat = np.concatenate(labels)
weights = class_weight.compute_class_weight('balanced',
                                            np.unique(labels_flat),
                                            labels_flat)
class_weights = dict(enumerate(weights))
class_name_weights = {np.unique(labels_flat)[key]:class_weights[key] for key in class_weights.keys()}
class_name_weights

/home/dhruv/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:68: FutureWarning: Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29], y=[ 6  6  6 ... 27 27 27] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


{0: 1.024599344652964,
 1: 1.6625966744006186,
 2: 0.4930025226464855,
 3: 1.5149665257223397,
 4: 0.10798359955796664,
 5: 0.85998099809981,
 6: 1.8468535223367697,
 7: 4.3232528908999495,
 8: 5.749882982280174,
 9: 3.420425616547335,
 10: 0.32435404171853194,
 11: 4.866411997736276,
 12: 1.3578004105479236,
 13: 2.7181760708076497,
 14: 5.84366292898403,
 15: 4.003235567970205,
 16: 1.7869804655029093,
 17: 8.778917815211843,
 18: 1.5418594226286535,
 19: 1.872185935118659,
 20: 3.476430159692743,
 21: 4.1966569058077114,
 22: 0.6601374174727468,
 23: 1.031049160671463,
 24: 17.061408730158732,
 25: 0.44120936914748965,
 26: 1.9412913421379387,
 27: 6.991016260162602,
 28: 2.9795391545391547,
 29: 0.9328939517222674}

In [7]:
ohe = OneHotEncoder()
ohe.fit(labels_flat.reshape((-1,1)))

labels = [ohe.transform(np.array(batch).reshape(-1,1)).toarray() for batch in labels]

NUM_CLASSES = len(weights)

# features = [batch.reshape((-1, sfreq*2, 1)) for batch in features]

# X_train = features
y_train = labels
del labels_flat

In [ ]:
len(y_train)

In [20]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, batch_size, num_batches, ohe, shuffle=True):
        'Initialization'
        self.indexes = np.arange(1, num_batches+1)
        self.batch_size = batch_size
        self.num_batches = num_batches
        self.ohe = ohe
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return num_batches

    def __getitem__(self, index):
        'Generate one batch of data'
        i = self.indexes[index]
        return self.__data_generation(i)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(1, num_batches+1)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, i):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        
        X_batch, y_batch = pickle.load(open('features/time_seg/train_batches/batch{}.pkl'.format(i), 'rb'))
        X_batch = X_batch.reshape((-1, sfreq*2, 1))[:,::4,:]
        y_batch = self.ohe.transform(y_batch.reshape((-1,1))).toarray()
 
        return X_batch, y_batch

In [21]:
train_batch_fn = os.listdir('features/time_seg/train_batches')
num_batches = len(train_batch_fn)
batch_size = 64

train_generator = DataGenerator(batch_size, num_batches, ohe, True) 

In [9]:
feature_mode = 'time_seg'
mode = 'test'

test_features = pickle.load(open('features/{}/{}_features.pkl'.format(feature_mode, mode), 'rb'))
test_labels = pickle.load(open('features/{}/{}_labels.pkl'.format(feature_mode, mode), 'rb'))

test_features = concat(test_features)
test_labels = concat(test_labels)

k=0
for i in range(len(test_features)-k):
    if test_labels[i-k] == []:
        del test_features[i-k]
        del test_labels[i-k]
        k+=1
        
test_features = np.concatenate(test_features).astype('float64')
test_labels = np.concatenate(test_labels).astype('int64')

test_labels = ohe.transform(test_labels.reshape((-1,1))).toarray()
test_features = test_features.reshape((-1, sfreq*2, 1))

X_test = test_features
y_test = test_labels

In [14]:
# X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

In [15]:
# print('X train shape {}'.format(X_train.shape))
print('X test  shape {}'.format(X_test.shape))
# print('y train shape {}'.format(y_train.shape))
print('y test  shape {}'.format(y_test.shape))

X test  shape (23657, 32000, 1)
y test  shape (23657, 30)


In [24]:
timesteps = sfreq * 2 // 4
input_dim = 1

i = Input(shape=(timesteps, input_dim))
# o = TCN(return_sequences=True, nb_filters=128)(i)
o = TCN(return_sequences=False, nb_filters=128)(i)
o = Dense(512, activation='relu')(o)
o = Dropout(0.4)(o)
o = Dense(NUM_CLASSES, activation='sigmoid')(o)
model = Model(inputs=[i], outputs=[o])

# model.summary()
tcn_full_summary(model)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])

checkpoint=ModelCheckpoint('checkpoints/model.h5', monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
earlystopping=EarlyStopping(monitor='val_loss', patience=20, verbose=1)
reducelr=ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_lr=0.000001) 

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 8000, 1)]         0         
_________________________________________________________________
matching_conv1D (Conv1D)     (None, 8000, 128)         256       
_________________________________________________________________
activation_20 (Activation)   (None, 8000, 128)         0         
_________________________________________________________________
conv1D_0 (Conv1D)            (None, 8000, 128)         384       
_________________________________________________________________
activation_18 (Activation)   (None, 8000, 128)         0         
_________________________________________________________________
spatial_dropout1d_12 (Spatia (None, 8000, 128)         0         
_________________________________________________________________
conv1D_1 (Conv1D)            (None, 8000, 128)        

In [ ]:
history = model.fit(X_test[:,::4,:], y_test, epochs=100, batch_size=64,
                    class_weight=class_weights, callbacks= [checkpoint, earlystopping, reducelr], shuffle=True)

In [ ]:
history = model.fit(train_generator,
                  validation_data=(X_test[:,::4,:], y_test),
                  class_weight=class_weights,
                    epochs=100,
                  callbacks=[checkpoint, earlystopping, reducelr],
                  shuffle=True,
                  use_multiprocessing=True,
                  workers=4)

Epoch 1/100
2687/2687 [==============================] - ETA: 0s - loss: 0.1346 - categorical_accuracy: 0.1258
Epoch 00001: val_categorical_accuracy improved from 0.15205 to 0.21118, saving model to checkpoints/model.h5
2687/2687 [==============================] - 1460s 543ms/step - loss: 0.1346 - categorical_accuracy: 0.1258 - val_loss: 0.1247 - val_categorical_accuracy: 0.2112
Epoch 2/100
2687/2687 [==============================] - ETA: 0s - loss: 0.1307 - categorical_accuracy: 0.1453
Epoch 00002: val_categorical_accuracy did not improve from 0.21118
2687/2687 [==============================] - 1461s 544ms/step - loss: 0.1307 - categorical_accuracy: 0.1453 - val_loss: 0.1291 - val_categorical_accuracy: 0.1632
Epoch 3/100
2687/2687 [==============================] - ETA: 0s - loss: 0.1284 - categorical_accuracy: 0.1555
Epoch 00003: val_categorical_accuracy improved from 0.21118 to 0.23139, saving model to checkpoints/model.h5
2687/2687 [==============================] - 1459s 543ms/s

2687/2687 [==============================] - ETA: 0s - loss: 0.1173 - categorical_accuracy: 0.1828
Epoch 00011: val_categorical_accuracy did not improve from 0.24014
2687/2687 [==============================] - 1460s 544ms/step - loss: 0.1173 - categorical_accuracy: 0.1828 - val_loss: 0.1294 - val_categorical_accuracy: 0.1886
Epoch 12/100
1790/2687 [==================>...........] - ETA: 7:44 - loss: 0.1158 - categorical_accuracy: 0.1853

In [ ]:
history = {
    'train': [],
    'test': []
}

max_acc = 0
for epoch in range(100):
    # Training
    num_batches = len(train_batch_fn)
    t = trange(num_batches, desc='epoch {}: Train'.format(epoch+1))

    for i in t:
        X_batch, y_batch = pickle.load(open('features/time_seg/train_batches/batch{}.pkl'.format(i+1), 'rb'))
        X_batch = X_batch.reshape((-1, sfreq*2, 1))
        y_batch = ohe.transform(y_batch.reshape((-1,1))).toarray()
        
        mini_batch_size = 8
        for a in range(batch_size//mini_batch_size):
            X = X_batch[a*mini_batch_size: (a+1)*mini_batch_size]
            y = y_batch[a*mini_batch_size: (a+1)*mini_batch_size]
            metrics = model.train_on_batch(X, y, class_weight=class_weights, reset_metrics=False, return_dict=True)
            t.set_postfix(metrics)
  
    history['train'].append(metrics)
    
    # Testing
#     X = X_test[:,::4,:]
    metrics = model.test_on_batch(X_test, y_test, reset_metrics=False, return_dict=True)
#     print("epoch {} val metrics: {}".format(epoch+1, metrics))
#     num_batches = len(X_test) 
#     t = trange(num_batches, desc='epoch {}: Test'.format(epoch+1))
#     for i in t:
#         X_batch = X_test[i]
#         y_batch = y_test[i]
#         num_samples = len(X_batch)
#         for j in range(num_samples):
#             X = X_batch[j:j+1]
#             y = y_batch[j:j+1]
#             lstm = model.get_layer('lstm')
#         lstm.reset_states()
    t.set_postfix("val metrics: {}".format(metrics))
    history['test'].append(metrics)
    
    if max_acc < metrics['categorical_accuracy']:
        print('val accuracy improved from {} to {}. saving model...'.format(max_acc, metrics['categorical_accuracy']))
        max_acc = metrics['categorical_accuracy']
        model.save('checkpoints/model_tcn.h5')
    print('')

In [ ]:
X_batch.shape

In [ ]:
plt.plot(history.history['val_categorical_accuracy'])

### Contextual Results

In [ ]:
model.load_weights('checkpoints/model_{}.h5'.format(sfreq))
print('Loaded best checkpoint!')

In [ ]:
labels_test = ohe.inverse_transform(y_test).flatten()

In [ ]:
pred = model.predict(X_test)

In [ ]:
np.unique(labels_test)

In [ ]:
encoding = {label: ohe.transform(np.array(label).reshape(-1,1)).toarray() for label in np.unique(labels_test)}

contexts = {
    'monolithic': ['alarm-clock', 'baby-cry', 'blender', 'car-horn', 'cat-meow',
       'chopping', 'cooking', 'cough', 'dishwasher', 'dog-bark', 'door',
       'doorbell', 'drill', 'engine', 'flush', 'hair-dryer', 'hammer',
       'hazard-alarm', 'knock', 'laugh', 'microwave', 'phone-ring', 'saw',
       'shaver', 'snore', 'speech', 'toothbrush', 'typing', 'vacuum',
       'water-running'],
    'bathroom': ['water-running', 'shaver', 'toothbrush', 'flush', 'hair-dryer'],
    'kitchen': ['hazard-alarm', 'speech', 'chopping', 'water-running', 'microwave', 'blender', 'dishwasher', 'cooking'],
    'bedroom': ['speech', 'baby-cry', 'cough', 'snore', 'alarm-clock'],
    'office': ['phone-ring', 'speech', 'cough', 'door', 'knock', 'typing'],
    'entrance': ['speech', 'door', 'knock', 'doorbell', 'laugh'],
    'workshop': ['drill', 'hazard-alarm', 'speech', 'vacuum', 'hammer', 'saw'],
    'outdoor': ['dog-bark', 'hazard-alarm', 'speech', 'car-horn', 'engine', 'cat-meow']
}

context_encoding = {}
for context in contexts.keys():
    context_encoding[context] = np.zeros((1,30))
    for activity in contexts[context]:
        context_encoding[context] += encoding[activity]    

In [ ]:
for context in contexts.keys():
#     context = list(contexts.keys())[0]
    valid_idx = [i for i, label in enumerate(labels_test) if label in contexts[context]]
    valid_pred = pred[valid_idx]
    valid_labels = labels_test[valid_idx]
    valid_pred = np.argmax(valid_pred * context_encoding[context], axis=1)
    valid_pred = [ohe.categories_[0][pr] for pr in valid_pred]
    acc = accuracy_score(valid_labels, valid_pred)
    f1 = f1_score(valid_labels, valid_pred, average='weighted')
    conf_mat = confusion_matrix(valid_labels, valid_pred, normalize='true')
    label_names = np.unique(valid_labels)
    print('{}, {}, {}, {}'.format(sfreq, context, acc, f1))
    # fig, ax = plt.subplots(figsize=(13,10))
    # sns.heatmap(conf_mat, cmap='Blues', ax=ax, xticklabels=label_names, yticklabels=label_names, annot=False)
    # ax.set_title('sFreq: {}, Context: {}, Acc: {}'.format(sfreq,context,acc))
    # ax.set_xlabel('Predicted Label')
    # ax.set_ylabel('True Label')

### File-Level Metrics 

In [ ]:
mode = 'wild'
test_features = pickle.load(open('features/{}_features.pkl'.format(mode), 'rb'))
test_labels = pickle.load(open('features/{}_labels.pkl'.format(mode), 'rb'))

test_features_flat = []
for cls in test_features:
    for file in cls:
        if len(file) != 0:
            test_features_flat.append(file)

test_labels_ohe = []
for cls in test_labels:
    for file in cls:
        if len(file) != 0:
            file_labels = ohe.transform(np.array(file).reshape((-1,1))).toarray()
            test_labels_ohe.append(file_labels)

NUM_CLASSES = 30

X_test = [file.reshape((-1, vggish_params.NUM_FRAMES, vggish_params.NUM_BANDS, 1)) for file in test_features_flat] 
y_test = [file.reshape((-1, NUM_CLASSES)) for file in test_labels_ohe]

In [ ]:
model.load_weights('checkpoints/model.h5')
print('Loaded best checkpoint!')

In [ ]:
labels_test = np.array([ohe.inverse_transform(file).flatten() for file in y_test])

In [ ]:
pred = np.array([model.predict(file) for file in X_test])

In [ ]:
label_int = soundtransfer.label
label_inv = soundtransfer.label_inv

encoding = {label: ohe.transform([[label]]).toarray() for label in ohe.categories_[0]}

contexts = {
    'monolithic': ['alarm-clock', 'baby-cry', 'blender', 'car-horn', 'cat-meow',
       'chopping', 'cooking', 'cough', 'dishwasher', 'dog-bark', 'door',
       'doorbell', 'drill', 'engine', 'flush', 'hair-dryer', 'hammer',
       'hazard-alarm', 'knock', 'laugh', 'microwave', 'phone-ring', 'saw',
       'shaver', 'snore', 'speech', 'toothbrush', 'typing', 'vacuum',
       'water-running'],
    'bathroom': ['water-running', 'shaver', 'toothbrush', 'flush', 'hair-dryer'],
    'kitchen': ['hazard-alarm', 'speech', 'chopping', 'water-running', 'microwave', 'blender', 'dishwasher', 'cooking'],
    'bedroom': ['speech', 'baby-cry', 'cough', 'snore', 'alarm-clock'],
    'office': ['phone-ring', 'speech', 'cough', 'door', 'knock', 'typing'],
    'entrance': ['speech', 'door', 'knock', 'doorbell', 'laugh'],
    'workshop': ['drill', 'hazard-alarm', 'speech', 'vacuum', 'hammer', 'saw'],
    'outdoor': ['dog-bark', 'hazard-alarm', 'speech', 'car-horn', 'engine', 'cat-meow']
}

context_encoding = {}
for context in contexts.keys():
    context_encoding[context] = np.zeros((1,30))
    for activity in contexts[context]:
        context_encoding[context] += encoding[label_int[activity]]    

In [ ]:
for context in contexts.keys():
    valid_idx = [i for i, label in enumerate(labels_test) if label[0] in [label_int[x] for x in contexts[context]]]
    valid_pred = pred[valid_idx]
    valid_labels = labels_test[valid_idx]
    valid_pred = [file * context_encoding[context] for file in valid_pred]

    # Transform to per file level labels
    valid_pred = [np.argmax(np.sum(file, axis=0)) for file in valid_pred]
    valid_labels = [file[0] for file in valid_labels]

    acc = accuracy_score(valid_labels, valid_pred)
    f1 = f1_score(valid_labels, valid_pred, average='weighted')
    conf_mat = confusion_matrix(valid_labels, valid_pred, normalize='true')
    # label_names = np.unique(valid_labels)
    print('{},{},{}'.format(context, acc, f1))

In [ ]:
valid_idx